<a href="https://colab.research.google.com/github/arnabghorui/smai_project/blob/main/SMAI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TF IDF & MLP

Here we are given two articles and we have to check similarity between them,
There are many metrics of similarity, <br>
<br>
One of them is TF IDF ( Total Frequency * Inverse Document Frequency). In this metric we multiply the number of times a word is present in a document to the inverse of the number of the articles that the word appears in. The inverse part shows the overall importance of the word and the frequency part shows how important the given word is in the context of the present article. <br>
<br>


###Loading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
f = open('/content/drive/MyDrive/SMAI Proj/Copy of train_data.json')
train_data = json.load(f)
g= open('/content/drive/MyDrive/SMAI Proj/Copy of test_data.json')
test_data = json.load(g)

In [4]:
print(test_data[0].keys())

print(len(test_data))

dict_keys(['pair_id', 'title_1', 'title_2', 'text_1', 'text_2', 'tags_1', 'tags_2', 'meta_lang_1', 'meta_lang_2', 'score'])
4316


After Filtering out the english articles - we compare the two articles in a given element. 

For that we append the two articles with keys : text_1 and text_2 

In [5]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from string import punctuation


english_articles_train = []
english_articles_test = []


for d in train_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_train.append(d)

for d in test_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_test.append(d)



total_train_articles = []
total_test_articles = []

y_train = []
y_test = []

for eng in english_articles_train:
  total_train_articles.append(eng['text_1'])
  total_train_articles.append(eng['text_2'])
  y_train.append(eng['score'])

for eng in english_articles_test:
  total_test_articles.append(eng['text_1'])
  total_test_articles.append(eng['text_2'])
  y_test.append(eng['score'])


'''
art1 = art1.split()

art2 = art2.split()

for i in range(len(art1)):
    art1[i] = ''.join([letter for letter in art1[i] if letter not in punctuation])

for i in range(len(art2)):
    art2[i] = ''.join([letter for letter in art2[i] if letter not in punctuation])
'''

#########################################################



tf = TfidfVectorizer(min_df =2)

x_train = tf.fit(total_train_articles)
x_train = tf.transform(total_train_articles)
x_test = tf.transform(total_test_articles)


print(x_train.shape)

print(type(x_train))



X_text_1_train = []
X_text_2_train = []
X_text_1_test = []
X_text_2_test = []


print(x_test.shape)




for j in range(1287):
  X_text_1_train.append(x_train[2*j])
  X_text_2_train.append(x_train[2*j+1])

for j in range(236):
  X_text_1_test.append(x_test[2*j])
  X_text_2_test.append(x_test[2*j+1])



(2574, 25972)
<class 'scipy.sparse.csr.csr_matrix'>
(472, 25972)


In [6]:
from scipy.sparse import csr_matrix
 

X1_train = []
X2_train = []
X1_test = []
X2_test = []

for i in range(1287):
  X1_train.append(np.array(csr_matrix.todense(X_text_1_train[i])))
  X2_train.append(np.array(csr_matrix.todense(X_text_2_train[i])))

for i in range(236):
  X1_test.append(np.array(csr_matrix.todense(X_text_1_test[i])))
  X2_test.append(np.array(csr_matrix.todense(X_text_2_test[i])))


y_train = np.array(y_train)
y_test = np.array(y_test)

#scor1 = np.reshape(scor1,(1287,1))
X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)

#rows1 = np.reshape()

X1_train = X1_train.reshape(1287,25972)
X2_train = X2_train.reshape(1287,25972)
X1_test = X1_test.reshape(236,25972)
X2_test = X2_test.reshape(236,25972)



#Model

This is the Siamese implementation of the 

In [7]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, LSTM, Input, concatenate

input_1 = Input(shape=(25972, ))
dense_1 = Dense(1000, )(input_1)

input_2 = Input(shape=(25972, ))
dense_2 = Dense(1000, )(input_2)

merge = concatenate([dense_1, dense_2])

dense_3  = Dense(200,)(merge)
dense_4  = Dense(50,)(dense_3)
dense_all = Dense(1,)(dense_4)
#second_dense = Dense(2, activation='softmax')(merge)

model = Model(inputs=[input_1, input_2], outputs=dense_all)
optimizer = keras.optimizers.Adam()  #learning_rate=0.001
model.compile(loss='mse', optimizer = optimizer) #, metrics=['RootMeanSquaredError']
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 25972)]      0           []                               
                                                                                                  
 dense (Dense)                  (None, 1000)         25973000    ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 1000)         25973000    ['input_2[0][0]']                
                                                                                              

In [8]:
history = model.fit([X1_train, X2_train], y_train, epochs=10)

Epoch 1/10
41/41 [==============================] - 3s 15ms/step - loss: 2.0922
Epoch 2/10
41/41 [==============================] - 1s 15ms/step - loss: 0.5736
Epoch 3/10
41/41 [==============================] - 1s 15ms/step - loss: 0.1385
Epoch 4/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0521
Epoch 5/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0270
Epoch 6/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0254
Epoch 7/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0256
Epoch 8/10
41/41 [==============================] - 1s 14ms/step - loss: 0.0378
Epoch 9/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0409
Epoch 10/10
41/41 [==============================] - 1s 15ms/step - loss: 0.0377


In [9]:
y_pred = model.predict([X1_test,X2_test])

8/8 [==============================] - 0s 4ms/step


In [14]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.feature_selection import r_regression
print('The mean absolute error for the model is ', mean_absolute_error(y_test,y_pred))
print('The mean squared error for the model is ', mean_squared_error(y_test,y_pred))

The mean absolute error for the model is  1.1683223929445623
The mean squared error for the model is  1.8702846275621536


In [ ]:
print(y_pred.shape)

(236, 1)


In [13]:
from scipy.stats import pearsonr

print(pearsonr(y_test,y_pred))

(array([0.23978364601809363], dtype=object), 0.00020043981758582774)
